# Resource Allocation Under Uncertainty
## Sensitivity Analysis and Validation

**Student Name:** [Your Name]  
**Date:** [Date]  
**Chosen context:** [e.g. Marketing – ad spend across channels]

## Assignment Overview

You will choose a resource allocation problem in your field, create a small synthetic dataset (CSV), build an optimization model, run sensitivity analysis and validation, then respond to a **stakeholder change of mind** with an updated model and comparison. The deliverable is one notebook with clear documentation and an executive summary suitable to hand off to a manager.

**Learning Objectives:**
- Identify a resource allocation decision in your field
- Distinguish decision variables vs inputs; list 3 key parameters with uncertainty
- Perform sensitivity analysis (vary 3 parameters ±20%); identify most critical parameter
- Run one what-if scenario and compare to base
- Apply validation (sanity checks, robust vs fragile)
- Part 2: Respond to stakeholder change of mind; update model; comparison visualization; impact analysis
- Communicate results in an elegant final write-up


In [ ]:
# Install required packages (if needed in Colab)
# Skip if running locally and packages are already installed
%pip install pulp pandas matplotlib -q


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, value, LpStatus
import io

print("Libraries imported successfully!")


## Creating Your Synthetic Dataset

Because each student uses a different field, there is no single provided dataset. You will **create a synthetic (made-up) dataset** as **CSV** that fits your chosen context.

**Required size:** Exactly **5 records** (5 rows). Each record = one option to allocate to (e.g. one channel, one segment, one shift type, one player, one project).

**What your dataset must include:**
1. **Exactly 5 options** (5 rows) with one row per option.
2. **At least two numeric columns per option** (e.g. cost per unit, conversion rate, default rate). You need at least two so that you have two parameters to vary in sensitivity analysis (see Section 1.4 below).
3. **One overall constraint value** (e.g. total budget, total hours). This is the "limited resource" cap for the **whole scenario**—one number, not per row. You do **not** put it in the CSV. You get it from the language model (e.g. "budget = 10000") and set it in your notebook code (e.g. `budget_total = 10000`). This single number is your **third parameter** for sensitivity.

**Where do the "three parameters" come from?** Later in the assignment (Section 1.4 and Part 2 sensitivity) you will list **3 key parameters** and vary each by ±20%. Those three are: **(1)** and **(2)** two numeric inputs from your options table (e.g. cost per unit and conversion rate), and **(3)** the overall constraint (e.g. budget total). The CSV has only the two numeric columns; the constraint is **one number you set in code** (e.g. `budget_total = 10000`), not a column in the CSV.

**Steps (use a language model such as ChatGPT, Claude, or Copilot):**
1. Open a **new** conversation.
2. Say what decision you are modeling and your context (marketing, credit risk, staff, baseball, or generic). State: "I need a synthetic dataset with **exactly 5 records** (5 options) for a resource allocation optimization. I will use it in Python with pandas and PuLP. I need the data as **CSV only**."
3. Ask: "Please give me the data as a **CSV**. First row = column headers. Include **exactly 5 rows** (one per option). Include columns for: [list columns for your context, e.g. option name, cost per unit, conversion rate]. Also give me the single constraint value (e.g. total budget). Output only the CSV and the constraint value; no extra text."
4. Say: "Use realistic but simple numbers. The table must have **exactly 5 rows**—no more, no fewer."
5. Copy the CSV from the reply and paste it into the next code cell (as a string or by saving to `options_data.csv`). Load it with `pd.read_csv()` and set your constraint variable (e.g. `budget_total`).

See the **README** for example prompts by context (marketing, credit risk, staff, baseball, generic).


In [ ]:
# Paste your CSV here and load it into options_df; set your constraint value (e.g. budget_total)
# The CSV has only option names and numeric columns (e.g. cost_per_unit, conversion_rate).
# The constraint (e.g. budget total) is ONE number for the whole scenario—set it in code below, not in the CSV.
# Option A: CSV as a string (replace the triple-quoted string with your CSV from the LLM)
csv_string = """
option,cost_per_unit,conversion_rate
Channel_A,2.5,0.04
Channel_B,1.8,0.025
Channel_C,1.2,0.02
Channel_D,5.0,0.06
Channel_E,0.3,0.015
"""
options_df = pd.read_csv(io.StringIO(csv_string.strip()))
budget_total = 10000  # Replace with your constraint value from the LLM

# Option B: If you saved the CSV to a file, use:
# options_df = pd.read_csv('options_data.csv')
# budget_total = 10000  # your value

print(options_df)
print(f"\nConstraint (e.g. budget_total): {budget_total}")
print(f"Number of options (rows): {len(options_df)}")


**Data documentation (TODO):** In one sentence, state that you used a language model to generate this data and what it contains (number of options, constraint name and value).

[TODO: Your sentence here.]


---
## Part 1: Base Model and Parameter Identification

### 1.1 Decision Statement

**Instructions:** Write a clear decision statement in the form: "I need to decide how to allocate [resource] across [options] for [timeframe]."

**Your Decision Statement:**
[TODO: Write your decision statement here.]


### 1.2 Decision Variables vs Inputs

**Instructions:** List your decision variables (what you control) and your inputs (what you know or estimate). Briefly explain why each belongs in its category.

**Decision Variables:**
[TODO: List here.]

**Inputs:**
[TODO: List here.]

**Explanation:**
[TODO: One or two sentences.]


### 1.3 Objectives and Constraints

**Instructions:** State your objective (what you maximize or minimize) and your constraints (limits that must be respected). One or two sentences each.

**Objective:**
[TODO: Your objective.]

**Constraints:**
[TODO: Your constraints.]


### 1.4 Key Parameters and Uncertainty

**What are the "three parameters"?** They are the inputs you will later vary in sensitivity analysis (±20%). They are:

1. **Two numeric inputs from your options table** (your CSV columns that are numbers, e.g. cost per unit, conversion rate, default rate, benefit per unit). In the sample data above, these are `cost_per_unit` and `conversion_rate`.
2. **The overall constraint value** (one number for the **whole scenario**, e.g. budget total, total hours). It is **not** in the CSV and **not** per row. In the sample, you set it in code: `budget_total = 10000`. That one number caps total allocation across all options.

So you do **not** need three columns in your table. The constraint is **not** a column—it is one value for the whole scenario that you set in the notebook when loading the data (e.g. `budget_total = 10000`). Your three parameters are: two columns from the table + that one constraint value.

**Instructions:** List **3 key parameters** that have uncertainty. For each give: (1) name, (2) source (data / estimate / assumption), (3) why it might differ from reality. Use the table below (examples: "Cost per unit", "Conversion rate", "Budget total").

| Parameter | Source | Why it might differ from reality |
|-----------|--------|-----------------------------------|
| [TODO]    | [TODO] | [TODO]                            |
| [TODO]    | [TODO] | [TODO]                            |
| [TODO]    | [TODO] | [TODO]                            |


### 1.5 PuLP Model

**Instructions:** Build your allocation model: decision variables (e.g. amount per option), objective (e.g. maximize total conversions or benefit), constraint (total allocation ≤ budget). Solve and print the base solution.


In [ ]:
# Build and solve the base allocation model
# TODO: Adapt variable names and column names to match your options_df
n = len(options_df)
model = LpProblem("Resource_Allocation", LpMaximize)

# Decision variables: amount to allocate to each option (e.g. spend per channel)
alloc = [LpVariable(f"alloc_{i}", lowBound=0) for i in range(n)]

# Objective: e.g. maximize total conversions (spend * conversion_rate / cost_per_unit per option)
# Adapt to your columns (e.g. cost_per_unit, conversion_rate)
cost_col = [c for c in options_df.columns if 'cost' in c.lower()][0] if any('cost' in c.lower() for c in options_df.columns) else options_df.columns[1]
rate_col = [c for c in options_df.columns if 'rate' in c.lower() or 'conversion' in c.lower() or 'benefit' in c.lower()][0] if any('rate' in c.lower() or 'conversion' in c.lower() or 'benefit' in c.lower() for c in options_df.columns) else options_df.columns[2]
model += lpSum([alloc[i] * (options_df.iloc[i][rate_col] / options_df.iloc[i][cost_col]) for i in range(n)]), "Total_benefit"

# Constraint: total allocation <= budget_total
model += lpSum(alloc) <= budget_total, "Budget"

model.solve()
print(f"Status: {LpStatus[model.status]}")
print(f"Objective value: {value(model.objective):.2f}")
for i in range(n):
    opt_name = options_df.iloc[i].get(options_df.columns[0], f"Option_{i}")
    print(f"  {opt_name}: {value(alloc[i]):.2f}")


---
## Part 2: Sensitivity and One What-If

### 2.1 Sensitivity Analysis

**Instructions:** Vary each of your 3 key parameters by ±20% (or ±10% and ±20%). Re-solve the model each time and record how the solution (allocation or objective value) changes. Identify which parameter is **most critical** (changes the solution the most).

[TODO: Which parameter is most critical? Explain in 2–3 sentences.]


In [ ]:
# Sensitivity analysis: vary parameters and record solution changes
# TODO: Implement loops or repeated solves that vary each of your 3 parameters by ±20%
# Store results in a table or DataFrame, then display
# Example placeholder: you will replace this with your actual parameter names and logic
results = []
# ... your sensitivity runs ...
# print or display results


### 2.2 Sensitivity Visualization

**Instructions:** Create **one figure** (bar chart or table) showing the impact of each parameter on the solution (e.g. change in objective value or in allocation). Give the figure a clear title and axis labels.


In [ ]:
# Create sensitivity visualization (e.g. bar chart of impact per parameter)
# TODO: Use your sensitivity results; add title and axis labels
# plt.bar(...); plt.xlabel(...); plt.ylabel(...); plt.title(...); plt.show()


### 2.3 One What-If Scenario

**Instructions:** Run one alternative scenario (e.g. budget cut by 20%, or demand +20%). Compare the new solution to your base solution.

[TODO: How does the recommendation change? Is the base solution robust to this scenario? (2–3 sentences.)]


In [ ]:
# What-if: e.g. budget_total * 0.8; re-solve and compare to base
# TODO: Implement one what-if scenario and print comparison


---
## Part 3: Validation and Recommendation

### 3.1 Sanity Checks

**Instructions:** Perform 1–2 sanity checks on your base solution (e.g. Do allocations sum to the resource? Are any values negative? Is the outcome plausible?)

[TODO: List your checks and the result of each in one sentence.]


### 3.2 Robust vs Fragile

**Instructions:** Based on your sensitivity and what-if analysis: Is this solution **robust** or **fragile**? What should the decision-maker watch (e.g. which parameter)?

[TODO: Short paragraph (3–5 sentences).]


---
## Part 4: Stakeholder Change of Mind

A stakeholder has changed their mind. You must choose **one** of the following (or an instructor-defined alternative) and update your model accordingly.

- **Budget (or resource) cut** by 15–20%.
- **Minimum allocation** to one specific option (e.g. at least $X in one channel, or at least 2 shifts for one team).
- **Different objective** (e.g. minimize cost instead of maximize conversions).
- **New constraint** (e.g. no single option can receive more than 40% of the total; or a cap per option).

### 4.1 Scenario

**Which scenario did you choose?** State it in one sentence.

[TODO: Your sentence here.]


### 4.2 Update Model

**Instructions:** Update your PuLP model (new constraint or new objective). Solve and store the new solution (e.g. in a variable or DataFrame for comparison).


In [ ]:
# Update model with stakeholder change; solve and store new solution
# TODO: Implement (e.g. new budget, or new constraint, or new objective)
# allocation_part2 = ...


### 4.3 Comparison Visualization

**Instructions:** Create **at least one figure** comparing Part 1 (base) vs Part 2 (after stakeholder change): e.g. bar chart of allocation by option (before vs after), or a table with old vs new allocation and objective value. **Title and axis labels must be clear.** In the markdown below, **reference the figure** (e.g. "As shown in the chart below…").


In [ ]:
# Comparison visualization: Part 1 vs Part 2 allocation (or objective)
# TODO: Create figure with title and axis labels; refer to it in the next markdown cell
# plt.bar(...); plt.title(...); plt.show()


[TODO: Refer to your figure in 1–2 sentences, e.g. "The chart above shows…"]


### 4.4 Impact Analysis

**Instructions:** In 3–5 sentences: What changed in the recommendation? Why does it matter for the stakeholder? What would you tell them in a short briefing?

[TODO: Your impact analysis here.]


---
## Executive Summary

**Instructions:** Write a professional summary suitable for presenting to a decision-maker. Complete all subsections; no placeholder text or [TODO] in your final submission.

### Key Decision and Model
[TODO: What was allocated, across what options, objective/constraints (2–3 sentences).]

### Sensitivity Findings
[TODO: Which parameter mattered most; how solution responded to what-if (2–3 sentences).]

### Validation
[TODO: Sanity checks and robust/fragile conclusion (1–2 sentences).]

### Stakeholder Change (Part 2)
[TODO: What changed when the stakeholder changed their mind; how the recommendation shifted; what you would brief them (2–3 sentences).]

### Final Recommendation
[TODO: What to do and what to monitor (2–3 sentences).]

### Key Insights
[TODO: What you learned about parameters, sensitivity, validation, and responding to changing stakeholder needs (2–3 sentences).]


---
## Expectations: What to Check Before Submitting

Run through this checklist before you submit:

- **Is the Executive Summary there?** All subsections filled in (key decision and model, sensitivity findings, validation, stakeholder change, final recommendation, key insights). No placeholder text or "[TODO]" left.
- **Are there grammatical errors?** Proofread the markdown and narrative cells (especially the Executive Summary and impact analysis).
- **Are there formatting errors?** Headings, lists, and code output look correct. The notebook runs from top to bottom without errors.
- **Is this in a format you would hand off to a manager?** The narrative is professional and clear; a reader could use the Executive Summary without reading the code.
- **Are the graphics properly labeled and referenced in the text?** Every figure has a clear title and axis labels. In the markdown or summary, each figure is referred to (e.g. "As shown in the chart above…"). The reader knows what each graphic shows and why it matters.

The rubric used for grading is provided separately.
